In [1]:
import sys
sys.path.append("..")
import imagenet_load
from importlib import reload
reload(imagenet_load)
import tarfile
import numpy as np
import imread
%pylab inline
import imread
from imread import imread_from_blob
from imagenet_load import orient
import multigpu
reload(multigpu)
import filter_gen
import conv
reload(conv)
import gc
import logging
import dill

Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['imread']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
%time loader = imagenet_load.ImagenetLoader(10, classes_path="../classes", mmap_loc="/dev/shm/imagenet_train", n_procs=32)

There exists a matrix at /dev/shm/imagenet_train


CPU times: user 48 ms, sys: 360 ms, total: 408 ms
Wall time: 613 ms


In [3]:
%time X = loader.load_all()

Loaded 0 classes into mem
CPU times: user 48 ms, sys: 108 ms, total: 156 ms
Wall time: 3.27 s


In [4]:
fg = filter_gen.make_gaussian_filter_gen(1.0, patch_size=11)

In [5]:
def conv_mmap(*args,
         **kwargs):
         from importlib import reload
         import conv
         import multigpu
         reload(conv)
         reload(multigpu)
         sidx = kwargs["sidx"]
         eidx = kwargs["eidx"]
         mmap_in_loc = kwargs["mmap_in_loc"]
         mmap_in_shape = kwargs["mmap_in_shape"]
         X = np.memmap(mmap_in_loc, mode="r", dtype='uint8', shape=mmap_in_shape)
         mmap_out_shape = conv.conv_compute_output_shape(data=X, **kwargs)
         X = X[sidx:eidx,:,:,:]
         mmap_out_loc = kwargs["mmap_out_loc"]   
         X_out = np.memmap(mmap_out_loc, mode="r+", dtype='float32', shape=mmap_out_shape)[sidx:eidx,:,:,:]
         X_out_local, filters = conv._conv(X,
                      filter_gen=kwargs["filter_gen"],                                                                                                                                                                         
                      num_feature_batches=kwargs["num_feature_batches"],                                    
                      feature_batch_size=kwargs["feature_batch_size"],
                      data_batch_size=kwargs["data_batch_size"],
                      pool_type=kwargs["pool_type"],
                      pool_size=kwargs["pool_size"],
                      pool_stride=kwargs["pool_stride"],
                      pad=kwargs["pad"],
                      bias=kwargs["bias"],
                      patch_size=kwargs["patch_size"],
                      conv_stride=kwargs["conv_stride"])
         np.copyto(X_out, X_out_local)
         X_out.flush()
         return multigpu.MmapResult(mmap_out_loc, 'float32', mmap_out_shape, "r+", range(sidx,eidx))

In [6]:
gpu = multigpu.GpuHandler(0)


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5105)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [7]:
conv_args = \
{ 
  "sidx":0,
  "eidx":4096,
  "mmap_in_loc":"/dev/shm/imagenet_train",
  "mmap_in_shape":X.shape,
  "mmap_out_loc": "/dev/shm/xout",
  "filter_gen":fg,
  "num_feature_batches":2,
  "data_batch_size":128,
  "feature_batch_size":256,
  "pool_size":32,
  "pool_type":"avg",
  "pool_stride":32,
  "patch_size":11,
  "pad":0,
  "bias":1,
  "conv_stride":4
}


In [8]:
mmap_out_shape = conv.conv_compute_output_shape(data=X, **conv_args)
